In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

# 1. データ準備
## 日経225連動型上場投資信託 1321

In [3]:
import os

# file一覧を作成
os.chdir('C:/Users/localadmin/Documents/data/1321')
file_list = os.listdir() 

# ファイル一覧を読み込みひとつのＤａｔａＦｒａｍｅにする
data1321 = pd.DataFrame([])
for i in file_list :
    tmp = pd.read_csv(i,encoding='shift-jis', skiprows=1, index_col='日付', parse_dates=True)
    #data1321 = pd.concat([data1321,tmp],ignore_index=True)
    data1321 = pd.concat([data1321,tmp])
    
data1321.tail()
#data1321.dtypes

,始値,高値,安値,終値,出来高,終値調整値
日付,,,,,,
2018-08-21,22580,22770,22520,22700,168314,22700
2018-08-22,22700,22850,22620,22810,80965,22810
2018-08-23,22870,22930,22850,22880,110272,22880
2018-08-24,22950,23070,22910,23060,218600,23060
2018-08-27,23150,23310,23150,23290,262550,23290


## 外国為替

In [4]:
import os

# file一覧を作成
os.chdir('C:/Users/localadmin/Documents/data/rate_exchange')
file_list = os.listdir() 

# ファイル一覧を読み込みひとつのＤａｔａＦｒａｍｅにする
exchange = pd.DataFrame([])
for i in file_list :
    tmp = pd.read_csv(i,encoding='shift-jis',  index_col='日付', parse_dates=True)
    exchange = pd.concat([exchange,tmp])

#exchange.round().astype(int)
#exchange.info()

exchange.tail()

,米ドル,英ポンド,ユーロ,カナダドル,スイスフラン,スウェーデンクローネ,デンマーククローネ,ノルウェークローネ,オーストラリアドル,ニュージーランドドル,...,バーレーンディナール,香港ドル,インドルピー,フィリピンペソ,シンガポールドル,タイバーツ,クウェートディナール,サウジアラビアリアル,UAEディルハム,メキシコペソ
日付,,,,,,,,,,,,,,,,,,,,,
2018-08-24,111.48,142.76,128.66,85.13,112.99,12.20,17.25,13.30,80.82,73.96,...,295.39,14.20,1.59,2.09,81.21,3.39,368.29,29.73,30.36,5.87
2018-08-27,111.25,143.03,129.41,85.51,113.21,12.21,17.35,13.34,81.61,74.47,...,295.09,14.17,1.59,2.08,81.51,3.41,366.80,29.66,30.29,5.92
2018-08-28,111.23,143.42,130.01,85.85,113.56,12.24,17.43,13.37,81.75,74.58,...,295.04,14.17,1.59,2.09,81.65,3.42,367.70,29.66,30.28,5.92
2018-08-29,111.23,143.05,129.94,86.01,113.84,12.17,17.43,13.33,81.64,74.65,...,294.73,14.17,1.59,2.09,81.54,3.41,367.95,29.66,30.29,5.84
2018-08-30,111.79,145.60,130.75,86.53,115.12,12.22,17.54,13.41,81.67,74.94,...,296.53,14.24,1.59,2.09,81.90,3.42,369.80,29.81,30.44,5.90


## 結合（ターゲットデータの作成）

In [95]:
target = pd.merge(data1321, exchange, right_index=True, left_index=True, how='outer')

#NaNを削除(行列の両方)
target2 = target.dropna().dropna(axis=1) 
target2.tail(1)

,始値,高値,安値,終値,出来高,終値調整値,米ドル,英ポンド,ユーロ,カナダドル,...,バーレーンディナール,香港ドル,インドルピー,フィリピンペソ,シンガポールドル,タイバーツ,クウェートディナール,サウジアラビアリアル,UAEディルハム,メキシコペソ
日付,,,,,,,,,,,,,,,,,,,,,
2018-08-27,23150.0,23310.0,23150.0,23290.0,262550.0,23290.0,111.25,143.03,129.41,85.51,...,295.09,14.17,1.59,2.08,81.51,3.41,366.8,29.66,30.29,5.92


## 訓練データ・テストデータの準備

In [111]:
from sklearn.model_selection import train_test_split

# テストデータの割合
test_rate = 0.1

# 問題データ(X)の準備
x_set = target2.iloc[0:len(target2)-1]
x_train_set, x_test_set = train_test_split(x_set, test_size=test_rate)

# 回答データ(Y)の準備
dif_set = target2.copy()[['始値','終値']]
dif_set['dif'] = target2['終値'] - target2['始値']

y_set = []
for s in range(0, len(dif_set)-1):

    if dif_set['dif'].iloc[s]> 0:
        y_set.append(1)
    else:
        y_set.append(-1)

#y_set = dif_set.iloc[1:len(dif_set)]
y_train_set, y_test_set = train_test_split(y_set, test_size=test_rate)

# 2. トレーニング

In [85]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=len(x_train_set), random_state=0)
rf.fit(x_train_set, y_train_set)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3622, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

# 3. テスト

In [86]:
# 予測結果の取得
y_predict = rf.predict(x_test_set)

In [99]:
def rate_check(answer, predict):
    count = 0
    for s in range(0, len(y_test_set)-1):
        if answer[s] == predict[s]:
            count+=1
        
    rate = count / len(answer) * 100
    print(rate)

In [115]:
rate_check(y_test_set, y_predict)

51.86104218362283


In [114]:
y_predict = rf.predict(x_train_set)
rate_check(y_train_set, y_predict)

5.5770292655991165


In [ ]:
 #Grid search

 from sklearn.grid_search import GridSearchCV

 # use a full grid over all parameters
 param_grid = {"max_depth": [2,3, None],
              "n_estimators":[50,100,200,300,400,500],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

 forest_grid = GridSearchCV(estimator=RandomForestClassifier(random_state=0),
                 param_grid = param_grid,   
                 scoring="accuracy",  #metrics
                 cv = 3,              #cross-validation
                 n_jobs = 1)          #number of core
    
forest_grid.fit(x_train_set, y_train_set) #fit

 forest_grid_best = forest_grid.best_estimator_ #best estimator
 print("Best Model Parameter: ", forest_grid.best_params_)